# Team 1 CS 105 Final Project

In [29]:
import os
import json
import pickle

id_tags_dict = {}
id_title_dict = {}

directory = 'E:\CS105-Final-Project\lastfm_subset'

for root, subdirectories, files in os.walk(directory):
    for file in files:
        with open(os.path.join(root, file)) as json_file:
            data = json.load(json_file)
            if data['tags']:
                id_tags_dict[data['track_id']] = [a[0].lower() for a in data['tags']]
                id_title_dict[data['track_id']] = data['title']

with open('id_tags_dict.pkl', 'wb') as f: 
    pickle.dump(id_tags_dict, f)
with open('id_title_dict.pkl', 'wb') as f: 
    pickle.dump(id_title_dict, f)

In [30]:
id_tags_dict = {}
id_title_dict = {}

with open('id_tags_dict.pkl', 'rb') as f: 
    id_tags_dict = pickle.load(f)
with open('id_title_dict.pkl', 'rb') as f: 
    id_title_dict = pickle.load(f)

In [31]:
id_genre_dict = {}
genre_totals = {'rock': 0, 'pop': 0, 'hip-hop': 0, 'country': 0, 'electronic': 0, 'metal': 0, 'alternative': 0}

def get_genre(tag_list):
    genre_counts = {'rock': 0, 'pop': 0, 'hip-hop': 0, 'country': 0, 'electronic': 0, 'metal': 0, 'alternative': 0}
    for tag in tag_list:
        for genre in genre_counts.keys():
            if genre in tag:
                genre_counts[genre] += 1
    return max(genre_counts, key=genre_counts.get)

for key, value in id_tags_dict.items():
    genre = get_genre(value)
    id_genre_dict[key] = genre
    genre_totals[genre] += 1

with open('id_genre_dict.pkl', 'wb') as f: 
    pickle.dump(id_genre_dict, f)

The code below shows the number of songs in each genre.

In [32]:
for key, value in genre_totals.items():
    print(key, ' : ', value)

rock  :  3408
pop  :  502
hip-hop  :  205
country  :  208
electronic  :  181
metal  :  299
alternative  :  30


In [44]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

vectorizer = CountVectorizer()
vectors = vectorizer.fit_transform(id_title_dict.values()).toarray()
df = pd.DataFrame(vectors, columns = vectorizer.get_feature_names_out(), index = id_title_dict.keys())

def genre(row):
    return id_genre_dict[row.name]

df['Genre'] = df.apply(genre, axis=1)
df.head()

,00,000,02,05,100,1000,10000,1001,11,11th,...,år,æons,ça,éducation,étape,étoiles,être,öron,żółte,Genre
TRACCUS128F92E1FEB,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,rock
TRACCVZ128F4291A8A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,metal
TRACCMO12903CA4168,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,rock
TRACCVS12903D014F8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,metal
TRACCJA128F149A144,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,rock
